In [10]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00a 0:00:01


In [21]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
import shutil

In [ ]:
print("Searching for dataset...")
found_path = None
for root, dirs, files in os.walk('/kaggle/input'):
    if 'n01440764' in dirs: 
        found_path = root
        print(f"Found Input Path: {found_path}")
        break

if not found_path:
    for root, dirs, files in os.walk('/kaggle/input'):
        if 'train' in dirs:
            found_path = os.path.join(root, 'train')
            print(f"Using fallback path: {found_path}")
            break


Searching for dataset...
Found Input Path: /kaggle/input/imagenet100/train.X1


In [23]:
INPUT_ROOT = found_path
OUTPUT_ROOT = '/kaggle/working/imagenet100_adversarial/train'

In [24]:
BATCH_SIZE = 64
EPSILON = 8/255
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Hardware: {device}")

if os.path.exists(OUTPUT_ROOT):
    shutil.rmtree(OUTPUT_ROOT) 
os.makedirs(OUTPUT_ROOT, exist_ok=True)

Hardware: cuda


In [ ]:
classes = [d for d in os.listdir(INPUT_ROOT) if os.path.isdir(os.path.join(INPUT_ROOT, d))]
classes.sort()
print(f"Found {len(classes)} classes. Creating target folders...")

for cls in classes:
    os.makedirs(os.path.join(OUTPUT_ROOT, cls), exist_ok=True)

# standard pre-trained ResNet18 as the victim model
model = models.resnet18(weights='IMAGENET1K_V1')

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model = model.to(device)
model.eval()

Found 25 classes. Creating target folders...


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 236MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(), 
])

dataset = datasets.ImageFolder(root=INPUT_ROOT, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

classifier = PyTorchClassifier(
    model=model,
    loss=nn.CrossEntropyLoss(),
    input_shape=(3, 224, 224),
    nb_classes=1000, 
    clip_values=(0, 1),
    device_type='gpu',
    preprocessing=(np.array([0.485, 0.456, 0.406]), np.array([0.229, 0.224, 0.225]))
)

In [27]:
attack = ProjectedGradientDescent(
    estimator=classifier,
    eps=EPSILON,
    eps_step=2/255,
    max_iter=10, 
    verbose=False
)

print(f"Starting generation on {len(dataset)} images...")
total_generated = 0

for images, labels in tqdm(dataloader):
    x_batch = images.cpu().numpy()
    x_adv = attack.generate(x=x_batch)
    
    for i in range(len(x_adv)):
        label_idx = labels[i].item()
        class_name = classes[label_idx]
        
        img_arr = (x_adv[i] * 255).astype(np.uint8)
        img_arr = np.transpose(img_arr, (1, 2, 0))
        img_pil = Image.fromarray(img_arr)
        
        save_path = os.path.join(OUTPUT_ROOT, class_name, f"adv_{total_generated}.png")
        img_pil.save(save_path)
        total_generated += 1

print("Generation Complete.")

Starting generation on 32500 images...


100%|██████████| 508/508 [23:40<00:00,  2.80s/it]

Generation Complete.


In [28]:
print("Zipping dataset...")
shutil.make_archive('imagenet100_adversarial', 'zip', '/kaggle/working/imagenet100_adversarial')
print("Done! Download 'imagenet100_adversarial.zip' from Output.")

Zipping dataset...
Done! Download 'imagenet100_adversarial.zip' from Output.
